<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Neural Signals and Signal Processing (NX-421)</h2>
<hr style="clear:both"></hr>

Welcome to the **OPTIONAL** notebook on advanced preprocessing.

It is *purely optional*. If you wish to understand most other preprocessing steps involved in fMRI pre-processing, you've come to the right place!

In [ ]:
# General purpose imports to handle paths, files etc
import os
import os.path as op
import glob
import pandas as pd
import numpy as np
import json


# Useful functions to define and import datasets from open neuro
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt



def reset_overlays():
    """
    Clears view and completely remove visualization. All files opened in FSLeyes are closed.
    The view (along with any color map) is reset to the regular ortho panel.
    """
    l = frame.overlayList
    while(len(l)>0):
        del l[0]
    frame.removeViewPanel(frame.viewPanels[0])
    # Put back an ortho panel in our viz for future displays
    frame.addViewPanel(OrthoPanel)
    
def mkdir_no_exist(path):
    if not op.isdir(path):
        os.makedirs(path)
        
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

### 2.1 Field map unwarping

The field itself it not homogeneous, as you've seen in class. This means, in turn, that there are distortions in the acquisition.
We can try to correct for it, through field maps, provided they've been acquired.

Fortunately this is the case in our dataset - but you will need to download them as we have avoided loading them for you - on purpose!

To make sure you've understood how to load datasets, here is the dataset of interest: https://openneuro.org/datasets/ds004226/versions/1.0.0

<img src="imgs/dataset_screen.png">

Your first task is to load:
- Subject 001 fieldmap files located in the fmap subfolder (WITH the JSON sidecars!)

You are free to use either the openneuro cell or resort to the direct download option.
Choose whichever is more convenient for you, download-time wise in particular.

Out of convenience, we already provide you with the openneuro-py case. Modify the command-line run below to target *all files in the fmap subfolder of sub-01*. As the command is, it would include all files in the *anat subfolder of sub-01*.

In [ ]:
dataset_fmap = 'ds004226'
subject = '001'
sample_path = "dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_fmap)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

fmap_path = op.join(bids_root, 'sub-001', 'fmap')

# Change the below command to include all files of the fmap subdirectory
cmd_to_modify = """openneuro-py download --dataset {} 
          --include sub-{}/anat/*
          --target_dir {}""".format(dataset_fmap, subject, bids_root).replace("\n", " ")

os.system(cmd_to_modify)

Again, remember this download might not be finished immediately :)
Now, assuming it *is*, let's have a look at what we have:

In [ ]:
print_dir_tree(bids_root, max_depth=5)

You can see that there are four files, corresponding to two fieldmap acquisitions. One is PA, the other is AP.

We need some parameters to be able to exploit these files.
In particular, we need to figure out:
- The phase encoding direction
- The total readout time

Your task is to figure out which keys to exploit for this.
Have a look at the code below (and feel free to play around a bit of course!) to setup the values properly.
To help you, we've loaded one of the two JSON sidecars:

In [ ]:
get_json_from_file(op.join(fmap_path, 'sub-001_acq-task_dir-{}_epi.json'.format('AP')))

In [ ]:
preproc_fmap_path = op.join(preproc_root, 'sub-001', 'fmap')
mkdir_no_exist(preproc_fmap_path)
direction_file = op.join(preproc_fmap_path, 'datain.txt')

f = open(direction_file, 'w')

for name in ['AP', 'PA']:
    data = get_json_from_file(op.join(fmap_path, 'sub-001_acq-task_dir-{}_epi.json'.format(name)))
    phase_dir = data['PhaseEncodingDirection'] # Extract here the phase encoding direction !
    total_readout_time = data['TotalReadoutTime'] # Extract here the total readout time !
    
    # We expect a specific format, namely x y z total_readout_time, where x,y and z are set to 1/-1 iff they are the phase
    # encoding direction, 0 otherwise.
    phase = [0, 0, 0, total_readout_time]
    is_neg = len(phase_dir) == 2 and phase_dir[1] == '-'
    phase_dir = phase_dir[0]
    phase[ord(phase_dir)-ord('i')] = -1 if is_neg else 1
    for i in range(3):
        f.write('{} {} {} {}\n'.format(phase[0], phase[1], phase[2], phase[3]))
f.close()

#### Creating the field map
Now, we will create the field map.
This process is tedious, sometimes hard to get right. We've provided you with a function that performs all these steps. Have a look through it and apply it.

In [ ]:
def generate_fmap_AP_PA():
    merged_phase_imgs = op.join(preproc_fmap_path, 'sub-001_acq-task_dir-fmap_merged')
    # Combine AP and PA as single file
    cmd = 'fslmerge -t {} {} {}'.format(merged_phase_imgs,
                                       op.join(fmap_path, 'sub-001_acq-task_dir-AP_epi.nii.gz'),
                                       op.join(fmap_path, 'sub-001_acq-task_dir-PA_epi.nii.gz'))
    os.system(cmd)
    
    # Now we generate the fieldmap
    output_fmap = op.join(preproc_fmap_path, 'fieldmap_ex')
    unwarped_img = op.join(preproc_fmap_path, 'se_epi_unwarped')
    cmd = 'topup --imain={} --datain={} --config={} --fout={} --iout={} -v'.format(
        merged_phase_imgs,
        direction_file, 
        'b02b0.cnf', 
        output_fmap, 
        unwarped_img)
    os.system(cmd)
    
    # Convert fmap units
    fslmaths(output_fmap).mul(6.28).run(output_fmap + '_rads')
    # Create magnitude fmap
    fslmaths(unwarped_img).Tmean().run(output_fmap + '_mag')
    
    # Extract fmap brain using bet
    bet(output_fmap + '_mag', output_fmap + '_mag_brain', robust=True, mask=True)

In [ ]:
generate_fmap_AP_PA()

In the step above, go and check all the steps. In particular, you might want to check first that the generated brain extracted fieldmap is correct and if not correct the brain mask and mask again the fieldmap, as you've done in the anatomical section.
<br><br>

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    Easy? Well, not always. Field maps for this dataset came in a specific format. But they can come in *many* different ways, meaning you will need to be very careful when recovering them. The steps outlined above in particular are only applicable in the case of having an AP-PA acquisition. Here is the full resource of FSL's FUGUE on field map unwarping: <a href=https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FUGUE/Guide>https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FUGUE/Guide</a> . Don't be afraid to refer to it, should you have a different format in a project!</p>
</span>
</div>


It now remains to apply the fieldmap! 

To do so we will apply to the **first volume of our series to show you the result of distortion correction**.
Your first task is thus to extract the first volume by modifying the below command:

In [ ]:
file_to_trim = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.nii.gz')
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'func'))
extracted_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_vol_1')

# Select only the FIRST volume!
start_vol = 0 # Where should we start? (First volume is 0, not 1 !)
number_of_volumes = 1 # How many volumes should we keep?

fslroi(file_to_trim, extracted_epi, str(start_vol), str(number_of_volumes))

In [ ]:
load(extracted_epi)

Great! Now, Run the command below to unwarp (correct the distortions) based on the fieldmap!

In [ ]:
fmap_path = op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads')
result= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_vol_1_fmap')

unwarpdir='y-'
dwell_time= data['DwellTime']
os.system('fugue -i {} --loadfmap={} --dwell={} --unwarpdir={} -u {}'.format(extracted_epi, fmap_path, dwell_time, unwarpdir, result))

Let's visualize!

In [ ]:
reset_overlays()
load(extracted_epi_path)
load(result)

You should observe the following two volumes:

<center>
<div style="display:inline-block;">
    <img src="imgs/uncorrected_brain.png" style="height: 200px;width:auto;border: blue 6px groove;" />
    <p style="text-align:center;">Before unwarp (sagittal)</p>
</div>
<div style="display:inline-block;">
    <img class="middle-img" src="imgs/corrected_brain.png"/ style="height: 200px;width:auto;border: green 6px groove;" />
    <p style="text-align:center;">After unwarp (sagittal)</p>
</div>
<br><br>
<div style="display:inline-block;">
    <img src="imgs/uncorrected_brain_2.png" style="height: 270px;width:250px;border: blue 6px groove;"/>
    <p style="text-align:center;">Before unwarp (axial)</p>
</div>
<div style="display:inline-block;">
    <img src="imgs/corrected_brain_2.png" style="height: 270px;width:250px;border: green 6px groove;" />
    <p style="text-align:center;">After unwarp (axial)</p>
</div>
</center>

Do you think the fieldmap made an improvement? To drive your answer, feel free to inspect both volumes in FSLeyes. Observe the frontal and ventral regions. Do you notice anything different? Which one seems to match better what you'd expect from the brain anatomy ? 


<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> Assessing quality of functional data 💡</b></p>
<p style='text-indent: 10px;'>
    When in doubt about what you observe, don't be afraid to go have a look at your T1 to compare against. If a structure in the functional shows up in the T1 but distorted, you'll find out faster this way.</p>
</span>
</div>

### 2.3 Slice time correction

The importance of this step is still being investigated in fMRI literature. See <a href="https://www.frontiersin.org/articles/10.3389/fnins.2019.00821/full">here</a> for an in-depth analysis of its impact on the pipeline. One of the take-aways from this reference is that slice-time correction together with motion correction does improve results of fMRI analysis and does not hurt.
Doing it before or after MC is not clear, as you can see in the reference above, so we're *choosing* here to showcase it after motion correction, but only time and further investigations will tell if there's a good order :)


#### 2.3.1- A toy example

To help you understand the underlying theory of slice-time correction, we will start from a rather unreasonable case on synthetic data, which will help you better visualize how slice-time correction affects the patterns.

As you'll see, this step can only be performed if you have knowledge of the way in which slices were acquired. Most of the time, fortunately, this is easy to recover. If it is not present in your data, you can ask the scanner operator to find out which sequence was used for your data acquisition.

Let's get started!

First load the file named "ground_truth.nii" in FSLeyes to visualize it. The data is 4D, go ahead a play a bit around to see exactly what happens during the sequence! (don't forget in case of flickering to untick the "Synchronize movie update" option of FSLeyes).

What you see is the ground truth, ie: the real phenomenon as it plays out!

We now have an acquisition sequence. It performs in slices, but not in a linear order.
Our sequence is even weirder: at a given time, not one but 9 slices are made at the same time! 

In [ ]:
%run generate_smileys.py

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline as Interp
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

def save_array_asnib(array, save_name):
    img = nib.Nifti1Image(array.astype(np.uint8), np.eye(4))
    nib.save(img, save_name)
    
def check_dims(axis_len, seq_len):
    if axis_len != seq_len:
        raise Exception('The number of slices in the sequence is different from the number of slices available in the axis. Are you sure this is the right axis?')

def reslice_with_timings(slice_dir, slice_sequence,input_data, original_times):
    assert(original_times.size == input_data.shape[3])
    
    n_acqs_per_tr = slice_seq.shape[0]
    n_multibands = slice_seq.shape[1]
    n_slices = slice_seq.size
    
    r= -1
    if slice_dir=='x':
        # For each slice in x, interpolate!
        n = input_data.shape[0]
        check_dims(n, n_slices)
        r=0
    elif slice_dir == 'y':
        # For each slice in y, interpolate!
        n = input_data.shape[1]
        check_dims(n, n_slices)
        r=1
    elif slice_dir == 'z':
        # For each slice in z, interpolate!
        n = input_data.shape[2]
        check_dims(n, n_slices)
        r=2    
    else:
        # Undefined yo
        raise Exception('Invalid dimension! Should be x, y or z')
    # Reshape the input data to have r as first dimension!
    input_data = np.swapaxes(input_data, 0, r)
    
    output_data = np.zeros(input_data.shape)
    print(output_data.shape)

    y_s = input_data.shape[1]
    z_s = input_data.shape[2]
    
    # Now, on the first axis, we will iterate over slices :)
    for b in range(0, n_acqs_per_tr):
        time_slice = original_times + b*1./n_acqs_per_tr
        # For all slices acquired together in the multiband
        slices = slice_seq[b]
        print('---------')
        print(time_slice)
        print(slices)
        for s in range(0, n_multibands):
            sl = slices[s]
            for y in range(0, y_s):
                for z in range(0, z_s):
                    lin_interper = Interp(time_slice, input_data[sl, y, z, :], k=1)
                    output_data[sl, y, z, :] = lin_interper(original_times)
    # Now that this very inefficient method is done we should remember to swap back the axis :)
    input_data =np.swapaxes(input_data, r, 0)
    output_data=np.swapaxes(output_data, r, 0)

    output_data[output_data < 0] = 0
    return output_data

In [ ]:
slice_seq = np.arange(0, 99).reshape((11, - 1))
slice_seq

As you can see, these slices are acquired in a sequential order that can be called either ascending or descending depending on your convention!
It means that the slices are obtained successively.

There are different types of slicing, which depends on your sequence. Notice that in our example we acquire several slices simultaneously (for example, slices 0 up to 9 are all acquired together!). This could be some multiband acquisition, for instance, but really it is mostly to help you visualize the effect of slice timing for the exercise. In practice the slices are defined to sample the signal in the most appropriate way, so our toy sequence will likely be too crude :)

In any case, we had some ground truth signal, that you can visualize in ground_truth_modulation.nii (don't forget to play the movie with the box unticked!)

This signal represents the true signal that we want to acquire.
The participant steps in an MRI, and the scanner operator uses the sequence we've defined above:

```
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62],
       [63, 64, 65, 66, 67, 68, 69, 70, 71],
       [72, 73, 74, 75, 76, 77, 78, 79, 80],
       [81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98]])
```

What this really means is that we acquire 9 slices at the same time and then move on:
(A gif to showcase what this means)

Your task will be two-folds:
1. Understand which axis the sequence was applied on, ie along which direction (X, Y or Z) was slicing performed
2. With this simple knowledge, apply a slice-timing correction algorithm 

This cute smiley will represent our neurological data! Every time point, the behaviour is simple: the smiley's intensity increases across time!
Here is what it looks like in FSLeyes:

In [ ]:
reset_overlays()
load('ground_truth_modulation.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

In [ ]:
load('ground_truth_upsampled_modulation.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

Using the following slice sequence, we have acquired our smiley signal.
The resulting timeseries is represented in the acquired_modulation file. 

In [ ]:
reset_overlays()
load('acquired_modulation.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

Oh no! What went wrong?

Well, if you think about it, nothing. It is simply that acquiring every slice takes time. During this time, the signal is evolving, so we're a little late in our acquisition, which causes the drift you're seeing.

This is what you observe in the acquired modulation file. The slice that is at the top - which is the last slice - also turns out to be the one with highest value. At timestep 0, it is in fact almost equal to 10 - the value of the ground truth at timestep 1 !

#### Correcting the delay

The heart of slice-timing correction is an interpolation in time.
Because the timing of the slices is wrong, we account for it by interpolating back to some reference time. This is how we obtain the resliced data.
For this, we need some informations. The first is the sequence in which slices are acquired, to know the lag of each slice. We also need the axis along which slices are acquired.

Based **only on the abnormal smiley visualization in FSLeyes** and the knowledge of the sequence (that is: a bottom-up sequence with 9 simultaneous bands in every slice), can you determine along with direction the acquisition was made?

- [ ] The X direction
- [ ] The Y direction
- [ ] The Z direction

Based on your answer, in the following cell, fill in the phase_encode variable, with either 'x', 'y' or 'z' (with the quotation marks!).

In [ ]:
phase_encode = ???

We will now conduct slice-timing correction: the idea is simply to interpolate back the slices in time along the slice direction. Easy right? Let's do it:

In [ ]:
smile_ts = nib.load('ground_truth_modulation.nii').get_fdata()
smile_resampled = nib.load('acquired_modulation.nii').get_fdata()
resliced_data = reslice_with_timings(phase_encode, slice_seq, smile_resampled, np.arange(0,9))
save_array_asnib(resliced_data.astype(np.uint8), 'resliced_data.nii')

reset_overlays()
load('resliced_data.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

Are you convinced on this toy example we did a not-too bad job?

#### 2.3.2  Application to real data

We have shown you the basic principle, but the application to real data requires some specific informations.
You need the following ingredients:
- When was each slice acquired in the sequence: **(Slice timing)**
- Along which axis were the slices acquired: **Phase direction**
- How much time we take to acquire all slices: **TR**

Let's go back to our practical dataset to extract these informations. Can you find them, when looking through the JSON sidecar?

In [ ]:
data = get_json_from_file(op.join(bids_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold.json'))
data

This data is actually a dictionary. We can thus extract the slice timing as an array directly from it. For example, to extract TaskName, we would use:
```python
data['TaskName']
```

Go ahead and extract the slice timing array, and store it in the slice_timing variable.

In [ ]:
slice_timing = data[???] # Replace with the appropriate key (have a look above!)

Now, we might want to know where our slices are, ie along which axis, right? Typically it is along the z-direction, but we're better off if we check! Using FSLeyes, determine how many slices each axis has **for the functional data of interest**. You should thus open the relevant functional file in FSLeyes to answer this question.


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>Using FSL command line</b></p>
<p style='text-indent: 10px;'>To figure out the dimensions of an MRI image, a faster option - if you have FSL installed directly - is to run the command line command:
    <blockquote>fslhd [your_volume]</blockquote>
This will give you all informations contained within the header of the NIfti file. For example, running the command for our volume will easily allow us to access the slice informations:
    <img src="imgs/fslhd_capture.png"></p>
</span>
</div>
Let's compare now with the amount of slices we have in our acquisition. We can consider simply the number of timings for this

In [ ]:
len(slice_timing)

So we have 42 slices in our slice timings, and you likely found 64 slices on the X axis, 64 axis on the Y axis and 42 slices on the Z axis. As a consequence, Z is the axis where the slices were acquired!
Great, so we know which axis we want, we know the slice timings, but we still need to know the TR. This information is also in the JSON sidecar! Extract it now!

In [ ]:
tr = data[???] # Extract the TR from the sidecar's appropriate field

To now perform the correction, we need to apply FSL's slicetimer command. For this, we need to save the timings first to their own separate file! Instead of giving the slice timings, we will provide instead the slice **order** (ie which slice was done in which order) and let FSL figure out how to best correct based on this information.

Let's do it.

In [ ]:
slice_order = np.argsort(slice_timing) + 1

# Write to a file the corresponding sorted timings :)
timing_path = op.join(preproc_root_demo,  'sub-01', 'func', 'sub-01_task-listening_run-1_slice-timings.txt')
file = open(timing_path, mode='w')
for t in slice_order:
    file.write(str(t) + '\n')
file.close()

Finally we can call slicetimer from a terminal!

In [ ]:
file_to_realign = op.join(bids_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold')
output_target = op.join(preproc_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold_slice-corr')
cmd = 'slicetimer -i ' + file_to_realign + ' -o ' + output_target + ' -r ' + str(tr) + ' -d 3 --ocustom=' + timing_path
os.system(cmd)

Had we launched it on the unscrubbed data, we would really notice the impact on the first volume. <span style="color:red;">Notice that you should in general **never** do this, as you would introduce lots of noise and garbage in your data.</span> Prefer to first clean all that is weird and then perform steps that might not bring a visible improvement rather than starting with data that is so bad that you can visually see changes when running above steps.
 <div class="row">
    <img src="imgs/slice_uncorr.png" style="width:100%">
      <center>First volume without slice correction</center>
    <img src="imgs/slice_corr.png" style="width:100%">
       <center>First volume with slice correction: the staircase has been more or less mitigated but the result is still imperfect...</center>
    <center>**And because of the linear interpolation, the garbage of volume 0 was spilled to volume 1!!!**</center>
    <img src="imgs/vol1_slice_uncorr.png" style="width:100%">
      <center>Second volume without slice correction</center>
    <img src="imgs/vol1_slice_corr.png" style="width:100%">
       <center>Second volume with slice correction: the result is worse than before...</center>
</div> 


The message here is: 
- **always** perform QC between your steps
- no algorithm can turn trash to gold. Remove the faulty volumes or you'll likely have a garbage-in garbage-out scenario. 

<u>Remember the 1 - 10 -100 dollar rule! It is much easier to avoid errors than compensate for them.</u>

## Where are we?

<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>Field unwarping</td><td style='text-align:justify;'>Correction distortions induced by inhomogeneities of the b0 field through maps acquired specifically to measure this field called fieldmaps.</td><td style='text-align:justify;'>FUGUE (but also FLIRT - see below)</td></tr>
    <tr><td></td><td style='text-align:left;'>Slice-timing correction</td><td style='text-align:justify;'>Accounting for the difference in acquisition between the slices that make up a volume to interpolate back voxels to a fixed time reference.</td><td style='text-align:justify;'>slicetimer</td></tr>
</table>

#### Including the fieldmap correction

If you've looked a bit, you've probably noticed some frontal "horns" in the axial view of the EPI.
While one could hypothesize perhaps the patient had some imp lineage, we notice that these horns are missing on the anatomical. What could explain these horns, based on all you've seen so far?
- [ ] A rare case of astral projection
- [ ] A distortion of the magnetic field

<br> And this is where fieldmaps come into play! For these to work, we need to modify our command in the following ways:
```python
epi_reg(..., wmseg=path_to_wm, fmap=path_to_fmap, fmapmag=path_to_fmap_magnitude, fmapmagbrain=path_to_bet_fmap_magnitude, echospacing=effective_echo_spacing, pedir=pahse_encoding_direction)
```

All these informations have been determined in the field map part of the tutorial, fortunately! Complete the following snippet to fill in all informations and get the coregistration running with fieldmap correction! (Remember: these informations should be extracted from the **EPI**, not the fieldmaps!!)

In [ ]:
output_path_fmap = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr_fmap')
dwell_time = ???
unwarpdir=???

In [ ]:
epi_reg(ref_vol_name, 
        whole_t1, 
        skull_stripped_t1, 
        output_path_fmap, 
        wmseg=op.join(preproc_root, 'sub-001', 'anat', 'T1_fast_pve_2'),
        fmap=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads'),
        fmapmag=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag'),
        fmapmagbrain=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag_brain'),
        echospacing=dwell_time,
        pedir=unwarpdir)

Now let's check whether this brought any improvement at all!

In [ ]:
load(output_path_fmap)

What do you think? Do you think it is any better? If so why, if so why not?

NOTE:
epi_reg is still using FLIRT under the hood! To be more specific, it is using flirt setup with its search cost as BBR. If you look through FLIRT's options, you'll notice that many more options are open to you:
<img src="imgs/flirt_options.png"/>

Feel free to explore the effect of different search costs :) But remember: not all costs are born equal when registering images **across modalities**!

## Applying the transformation to all volumes

You must have noticed that the coregistration has been done only on a single EPI volume, right?
This is, as you've guessed, because all volumes are aligned with respect to each other following motion-correction. As such, it is wasteful to compute more than once the transformation from EPI to anatomical.

The order of transformations we would like to have is:

<img src="imgs/transforms.png"/>

Notice that motion correction was applied by selecting a reference volume in the EPI. By default, the middle EPI was used. So, let's extract this EPI. It will be on this one that we will work, compute all transformations and reason.
**It is critical that you pay attention to which image was used to compute your transformations, otherwise combining them won't make sense!**.
For this reason, let's now go over the entire pipeline and transformation steps, sticking to this middle EPI. We extract it again with fslroi, as you've seen before:

In [ ]:
original_epi = op.join(bids_root_fmap, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
middle_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_middle-vol')
fslroi(original_epi, middle_epi, str(182), str(1))

Now, let's do motion correction. Recall that it is done on the **entire** EPI timeseries with mcflirt. We will explicitly give the middle epi as reference this time around, to force FSL to use this volume and realign everyone to it!

In [ ]:
path_moco_data = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
mcflirt(infile=original_epi,o=path_moco_data, plots=True, report=True, dof=6, mats=True, reffile=middle_epi)

Fantastic! Now, because **the reference volume did not move at all** (since it is the reference to which everyone is realigned), we can use this volume as starting point to compute our other transforms: we're only missing the coregistration with fieldmap unwarping, as the normalization is obtained through the anatomical data :)

In [ ]:
whole_t1 = op.join(preproc_root, 'sub-001', 'anat', 'T1_biascorr')
skull_stripped_t1 = op.join(preproc_root, 'sub-001', 'anat', 'T1_biascorr_brain')
output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space')
dwell_time = 0.000620007
unwarpdir='y-'

epi_reg(middle_epi, 
        whole_t1, 
        skull_stripped_t1, 
        output_path, 
        wmseg=op.join(preproc_root, 'sub-001', 'anat', 'T1_fast_pve_2'),
        fmap=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads'),
        fmapmag=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag'),
        fmapmagbrain=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag_brain'),
        echospacing=dwell_time,
        pedir=unwarpdir)

Beautiful! 
➡️ Inspect the two resulting files to ensure that nothing went wrong. In other words:
- Check that the MCFLIRT result is okay with respect to motion
- Check that the realignment following epi_reg made the EPI well aligned with the anatomical

Now, if you inspect the folder you'll see we have several files. Some of them are...Well garbage left-over by FSL, but others are more precious: they are transformation files, which describe the transforms we just did.
Namely:

--
```
sub-001_task-sitrep_run-01_bold_bbr_fmap_warp
```
corresponds to the anatomical + field unwarping transformation.
The \_warp was added by FSL, to signal that this is a nonlinear transformation (otherwise it would be a .mat file, but here this is a NIFTI!). But what sort of transformation is it? Where does it map?
It actually goes from EPI (after motion correction) to Anatomical space.
To convince yourself of this fact, you can apply the transformation yourself, using the applywarp function, whose signature is as follows:
```python
applywarp(volume_on_which_transform_is_applied, reference_volume, warp1=transformation_to_apply)
```
--
```
sub-001_task-sitrep_run-01_bold_bbr_fmap_warp
```
corresponds to the motion correction transforms we had (one per each volume!)


<u>Our goal</u>: now that we've computed all these transformations **for our reference volume**, we will combine them and apply all of them at every volume of the EPI!
This way, we will perform an interpolation of our data exactly once, after the last transform and thus minimize error propagation.

However we must tackle one problem: FSL provides us with a tool to apply transformations (even non linear ones such as fieldmap correction) per volume, but not in 4D.
We will thus:
- Split our 4D volume in individual 3D volumes
- Apply the transformation corresponding to each volume separately
- Combine back the volumes in a single 4D volume
- Remove intermediary volumes

The first step, to make everything clear, is to work on the transformations. Let's get started!


### Which transformations to combine?
    
Answer the following question as brain teaser:

- [ ] Knowing the MCFLIRT transform uses FLIRT and is done to perform motion-correction with 6 dof, is it a linear transformation represented by a matrix or a non linear transformation represented by a NIFTI volume?
- [ ] Assuming we have a total transformation EPI -> Standard space, should our reference be the anatomical volume in standard space or the starting EPI?
- [ ] Do we need one individual EPI (**before** moco) > Standard space transform per EPI volume?

Okay, now we will combine the three transformations as a single warp. To do so, we use the following code snippet:

In [ ]:
def combine_all_transforms(reference_volume, warp_save_name,  is_linear, epi_2_moco=None, epi_2_anat_warp=None, anat_2_standard_warp=None):
    """
    Combines transformation before motion correction all the way to standard space transformation
    The various transformation steps are optional. As such, the final warp to compute is based on 
    which transforms are provided.
    """
    args_base = {'premat': epi_2_moco, 'warp1': epi_2_anat_warp}
    if is_linear:
        args_base['postmat'] = anat_2_standard_warp
    else:
        args_base['warp2'] = anat_2_standard_warp
    args_filtered = {k: v for k, v in args_base.items() if v is not None}

    #print(args_filtered)
    convertwarp(warp_save_name, reference_volume, **args_filtered)

Now, to apply the transformation:

In [ ]:
def apply_transform(reference_volume, target_volume, output_name, transform):
    applywarp(target_volume,reference_volume, output_name, w=transform, rel=True)

Using these two functions should not be too hard now. Notice that in combine_all_transforms, setting any transform to None instead of the correct transform will skip the transform step in the total transformation. This way, you should be able to perform quality control. In particular, please ensure that:
- [ ] Applying ONLY motion correction transformation to the first volume yields the expected alignement (so it should be aligned with the \_moco volume.)
- [ ] Applying motion correction + epi -> anat should be aligned to anatomical
- [ ] Finally, applying motion correction + epi > anat + anat > standard should be aligned to the standard
Only once you're convinced these steps are working well should you proceed to standard space. **Remember the 1-10-100 rule! Always perform QC before moving on.**

To help you, we provide you below with the template to do such a thing, so that you don't have to worry too much about the nitty gritty details.
Focus on:
- The reference file to use 
- The transformations to provide (either a file or None)

<b>Notice this is performed only on a single volume. Indeed, if you are debugging you should avoid wasting time applying transformations on entire timeseries to quickly diagnose whether a step is working or failing.</b>

In [ ]:
import time
ref=op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin.nii.gz')
anat_2_mni= op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin_coeff.nii.gz')
func_2_anat= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr_fmap_warp.nii.gz')

# We show this one when selecting the middle EPI (volume 182)
middle_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_middle-vol')
split_nbr = '0182'

# We will name its warp as split0182 to show you how to do this for any volume
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_split' + split_nbr + '_epi_2_std_warp')

# Get the transformation matrix of this volume (this one is actually the unit matrix, 
# since this volume is the reference)
epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)

s0 = time.time()

# -- Step 1: Combine the transformations, that is :
#    EPI -> Motion correction -> Coregistration to anatomical -> Normalization to standard
#    We obtain a single warp, going from EPI -> Standard space
combine_all_transforms(ref, warp_name,  False, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)
s1 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)

# -- Step 2: Apply the transformation to our EPI
applywarp(middle_epi,ref, out_vol, w=warp_name, rel=True)
s2 = time.time()

Above, we've timed the steps to estimate which one might be more expensive, between combining the transforms and applying them. Let's check:

In [ ]:
print('Transform combination time:', s1 - s0)
print('Apply transform time:', s2 - s1)

As you can clearly see, combining the transforms is more than 100 times slower than applying the final transform. As a consequence, we would like to do this step as rarely as we can.

#### Optimizing a bit

Okay, so this step is slow. Can we make it faster? Well, yes!

Note that computing all these non linear fields <u>will</u> take time. We've seen above in fact that it is <u>the</u> most expensive step.
Now, applywarp has a neat option. It allows us to apply a transformation using a pre transformation matrix followed by the warp. Why is it cool?
Well, remember this picture?

What if we made it this way?

Then, we only compute one warp once, and we can call applywarp on the volumes themselves, which will be much quicker and is **strictly** equivalent! To do this, we must combine all transforms except the motion-correction transform, which is easily done:
```python
combine_all_transforms(..., epi_2_moco=None,...)
applywarp(split_vol, ref, out_vol, w=warp_name, rel=True, premat=epi_moco)
```

Let's compare the two methods, runtime wise:

In [ ]:
# ----- START OF METHOD 1 
# In this method, we compute the transform from start to finish and apply it
s0 = time.time()
combine_all_transforms(ref, warp_name,  False, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)
s1 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v1')
applywarp(middle_epi,ref, out_vol, w=warp_name, rel=True)
s2 = time.time()
# ----- START OF METHOD 2
# In this method, we compute the transform only post motion correction. We apply the motion correction and then the warp
combine_all_transforms(ref, warp_name,  False, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)
s3 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2')
applywarp(middle_epi,ref, out_vol, w=warp_name, premat=epi_moco, rel=True)
s4 = time.time()

print('Method 1 runtime:', s2 - s0, '({} for combination, {} to apply)'.format(s1 - s0, s2 - s1))
print('Method 2 runtime:', s4 - s2, '({} for combination, {} to apply)'.format(s3 - s2, s4 - s3))

To convince you that the two produced images are almost identical (you might notice differences on the order of the $10^{-3}$, but consider the relative error this entails and why such an error might happen):

In [ ]:
reset_overlays()
load(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v1'))
load(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2'))

Why does it matter? Well, just applying a back-of the envelope calculation, the first method takes 40s per volume, while the second method takes 25 seconds to combine **once** the transforms excluding motion correction, and 0.4 seconds per volume to apply the transforms including motion correction. If we plot the two with an increasing number of volumes, we can see why this quickly becomes relevant:

In [ ]:
%matplotlib inline
x = np.arange(0, 1000, 10)
plt.plot(x, x*40, label='Method 1')
plt.plot(x, 25 + x*0.3, label='Method 2')
plt.xlabel('Number of volumes')
plt.ylabel('Runtime (seconds) [LOG SCALE]')
plt.legend()
plt.yscale('log')
plt.show()

Hopefully, you're convinced that:
- We don't lose anything using method 2 imaging-wise
- We have a benefit in using method 2, computation-wise


### Applying it to the entire timeseries

With all this in mind, let's now apply our transformation to all our volumes! The steps are:

1. Split our EPI into all individual volumes (remember: applywarp only works on a single 3D image but our EPI is 4D).
2. Combine all transformations from EPI after motion correction all the way to standard space **once**. 
3. Use applywarp for every volume, passing the motion correction transform of this volume and the EPI > standard space warp
4. Combine back all volumes as a single 4D EPI in standard space

Ready? Let's go!

In [ ]:
# We will split our starting EPI volume across time 
split_target = original_epi
split_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_split')
cmd = 'fslsplit ' + split_target + ' ' + split_name + ' -t'

os.system(cmd)

In [ ]:
# Now, let's combine the different transforms EXCEPT motion correction!
ref=op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin')
anat_2_mni= op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin_coeff')
func_2_anat= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space_warp')
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_epi_moco_2_std_warp')

combine_all_transforms(ref, warp_name,  False, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)

# Here, we will apply transformation WITH motion correction to all our volumes
produced_vols = []

# Notice that we are sorting the volumes here! This is important, to make sure we don't get them in random order :)
split_vols = sorted(glob.glob(op.join(preproc_root, 'sub-001', 'func', '*_bold_split*')))
for split_vol in split_vols:
    split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
    epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)
    out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)
    applywarp(split_vol,ref, out_vol, w=warp_name, premat=epi_moco, rel=True)
    produced_vols.append(out_vol)

In [ ]:
total_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std')
tr = 1.5
cmd = 'fslmerge -tr ' + total_epi + ' ' + ' '.join(produced_vols) + str(tr)
os.system(cmd)

# Let's not forget to remove all the temporary volumes, shall we?
for v in split_vols + produced_vols:
    l = glob.glob(v + '*')
    for e in l:
        os.remove(e)

Now, let's check we did a proper job. If we did a proper mapping, we should definitely observe the EPI positioned on the anatomical in MNI space. How well did we do?

In [ ]:
reset_overlays()
load(ref)
load(total_epi)